# 1.x to 2.x Rule Migration Guide
This guide describes changes needed for rules to run under Insights Core 2.x. It covers the following topics:
- [@rule interface](#rule_interface)
- [function signatures](#component_signature)
- [filtering](#filtering)
- [cluster rules](#cluster_rules)
- [testing](#testing)
- [new style specs](#new_style_specs)

<a id="rule_interface"></a>

## @rule Interface
The `requires` keyword is gone, and required dependencies are no longer lists.
```python
@rule(requires=[InstalledRpms, PsAuxcww])
```
is now
```python
# requires InstalledRpms and PsAuxcww
@rule(InstalledRpms, PsAuxcww)

```
If a rule requires at least one of a set of dependencies, they are specified in a list like before.
```python
# requires InstalledRpms and at least one of ChkConfig or UnitFiles
@rule(InstalledRpms, [ChkConfig, UnitFiles])
```

And `optional` dependencies haven't changed.
```python
# requires InstalledRpms and PsAuxcww. Will use NetstatS if it's available
@rule(InstalledRpms, PsAuxcww, optional=[NetstatS])
```

<a id="component_signature"></a>

## Component Signature
The `local` and `shared` parameters are gone. Instead, component signatures should define parameters matching the dependencies in their `@rule` decorators.

```python
# Requires InstalledRpms and PsAuxcww.
@rule(InstalledRpms, PsAuxcww)
def report_thing(rpms, ps):
    pass

# Requires InstalledRpms and at least one of ChkConfig or UnitFiles.
# Both ChkConfig and UnitFiles may be populated, but only one of them is required.
# If one of them isn't available, None is passed as its value.
@rule(InstalledRpms, [ChkConfig, UnitFiles])
def report_something(rpms, cfg, uf):
    pass

# Requires InstalledRpms, at least one of ChkConfig or UnitFiles, and will use NetstatS
# if it's available. Notice how the order of report_something_else's parameter list
# matches the order of the dependencies even when the dependency specification is
# complicated.
@rule(InstalledRpms, [ChkConfig, UnitFiles], optional=[NetstatS])
def report_something_else(rpms, cfg, uf, netstat):
    pass
```

In [1]:
# Boilerplate used in later cells
# Not necessary for new rules.

from pprint import pprint

from insights.core import dr
from insights.core.filters import add_filter, get_filters
from insights.core.context import HostContext
from insights.core.plugins import make_response, rule

dr.load_components("insights.specs")
dr.load_components("insights.parsers")
dr.load_components("insights.combiners")


def run_component(component, broker=None):
    graph = dr.get_dependency_graph(component)
    if not broker:
        broker = dr.Broker()
        broker[HostContext] = HostContext(None)
    return dr.run(graph, broker=broker)

### @rule Example

In [2]:
from insights.parsers.installed_rpms import InstalledRpms
from insights.parsers.ps import PsAuxcww

@rule(InstalledRpms, PsAuxcww)
def report(rpms, ps):
    rpm_name = "google-chrome-stable"
    if rpm_name in rpms and "chrome" in ps:
        rpm = rpms.get_max(rpm_name)
        return make_response("CHROME_RUNNING",
                             version=rpm.version,
                             release=rpm.release,
                             arch=rpm.arch
                            )

In [3]:
broker = run_component(report)
pprint(broker[report])

{'arch': 'x86_64',
 'error_key': 'CHROME_RUNNING',
 'release': '1',
 'type': 'rule',
 'version': '61.0.3163.91'}


<a id="filtering"></a>

## Filtering
Filters are now applied to datasources instead of certain `Parser` classes.

```python
from insights.core.filters import add_filter

# like this
add_filter("messages", "KEEP_ME")
add_filter("messages", ["KEEUP_US", "KEEP_US_TOO"])

# instead of this
Messages.filters.append("KEEP_ME")
Messages.filters.extend(["KEEUP_US", "KEEP_US_TOO"])
```

<a id="cluster_rules"></a>

## Cluster Rules
TBD

<a id="testing"></a>

## Testing
Unit tests need to reflect the new rule function signatures.

`@archive_provider` calls should now pass the rule function instead of the rule module.

```python
from insights.plugins import vulnerable_kernel

# like this
@archive_provider(vulnerable_kernel.report)
def integration_tests():
...

#instead of this
@archive_provider(vulnerable_kernel)
def integration_tests():
...
```

<a id="new_style_specs"></a>

## New Style Specs
Specs in 2.x are called "data sources", and they're functions like rules and other components. However, they are special because they get passed an object called a `broker` instead of directly getting their dependencies, and they're meant to execute directly on the machine you want to analyze. The `broker` is like the `shared` object in 1.x.

In [4]:
from insights.core.plugins import datasource
from insights.core.spec_factory import TextFileProvider

@datasource()
def release(broker):
    return TextFileProvider("/", "etc/redhat-release")

In [5]:
broker = run_component(release)
print broker[release].content

['Fedora release 25 (Twenty Five)']


This allows data sources to generate content using the full power of python. Almost anything can go in the function body of a data source.

Directly defining data sources is powerful, but it's tedious when you just want to collect files or execute simple commands. The `SpecFactory` class streamlines those use cases by creating `@datasource` decorated functions for you.

In [6]:
from insights.core.spec_factory import SpecFactory

sf = SpecFactory()
hosts = sf.simple_file("/etc/hosts", name="hosts")
uptime = sf.simple_command("/bin/uptime", name="uptime")

print hosts
print uptime

<function hosts at 0x7f9a32fc5410>
<function uptime at 0x7f9a32fc5ed8>


Pass the `name` keyword to ensure the functions returned by `SpecFactory` have a sensible name and are attached to the defining module.

In [7]:
broker = run_component(hosts)
broker = run_component(uptime, broker=broker)

pprint(broker[hosts].content)
pprint(broker[uptime].content)

['127.0.0.1   localhost localhost.localdomain localhost4 localhost4.localdomain4',
 '::1         localhost localhost.localdomain localhost6 localhost6.localdomain6']
[' 16:53:12 up 11 days, 58 min,  1 user,  load average: 1.43, 0.82, 0.86']


Because new style specs are functions, they can't be used to both collect data from a machine and to load it from old archives. 2.x has a new system that doesn't need specs for handling data produced by datasources, and old style specs need to be maintained so long as old archives or sos reports need to be evaluated.

To allow parsers to depend on either new or old style specs, a datasource can use the `alias` keyword to give itself the same name for dependency resolution as the key of the old style spec that it replaces.

```python
# in insights/specs.py
hostname = sf.simple_command("/bin/hostname -f", name="hostname", alias="hostname")

# in insights/config/specs.py
"hostname"                  : CommandSpec("/bin/hostname"),
```

The `Hostname` parser can just depend on the string "hostname", and content will be associated with that key in one of three ways.
1. An old archive was evaluated using the old specs to search through its contents.
2. A new archive was evaluated using the new 2.x serialization system to reproduce a datasource's output.
3. The datasource executed in the same process as the Hostname parser.

Because the new spec system is more powerful than the previous one, we won't be able to express some new specs in the old way.